In [1]:
pip install pandas rdflib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from rdflib import Graph

In [3]:
# Leer el archivo CSV con clases y subclases
df_classes = pd.read_csv('FinalDataset/DBpediaClasses.csv', header=None)
df_classes.columns = ['Class', 'Subclass']

In [4]:
df_classes = df_classes.drop(0).reset_index(drop=True)
print(df_classes.head(10))

                  Class              Subclass
0                 Abbey     ReligiousBuilding
1       AcademicJournal  PeriodicalLiterature
2       AcademicSubject        TopicalConcept
3              Activity                  root
4                 Actor                Artist
5  AdministrativeRegion                Region
6            AdultActor                 Actor
7                 Agent                  root
8         Agglomeration        PopulatedPlace
9              Aircraft  MeanOfTransportation


In [5]:
# Leer el archivo TTL con las relaciones de clases
g_clases = Graph()
g_clases.parse('FinalDataset/instance_types_es.2017-04-01.ttl', format='ttl')

# Leer el archivo TTL con los textos
g_textos = Graph()
g_textos.parse('FinalDataset/long_abstracts_es.ttl.2017-04-01', format='ttl')

<Graph identifier=N9543027db87b4a3894a7d358e4c1ed51 (<class 'rdflib.graph.Graph'>)>

In [6]:
# Crear un diccionario para almacenar las relaciones
resource_to_class = {}
resource_to_text = {}

# Extraer las relaciones de clases
for s, p, o in g_clases:
    if str(p) == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        resource_to_class[str(s)] = str(o)
    #if i < 10:
        #print(str(s))
        #print(str(o))

# Extraer las relaciones de textos
for s, p, o in g_textos:
    if str(p) == 'http://dbpedia.org/ontology/abstract':
        resource_to_text[str(s)] = str(o)
    #if i < 10:
        #print(str(s))
        #print(str(o))

# Crear una lista para almacenar las filas del nuevo CSV
rows = []

for resource, clase in resource_to_class.items():
    clase = clase.split('/')[-1]
    if resource in resource_to_text:
        texto = resource_to_text[resource]
        # Verificar si la clase existe en el DataFrame
        subclase_series = df_classes.loc[df_classes['Class'] == clase, 'Subclass']
        if not subclase_series.empty:
            subclase = subclase_series.values[0]
            rows.append([clase, subclase, texto])

# Crear el DataFrame final y guardar en un archivo CSV
df_final = pd.DataFrame(rows, columns=['Class', 'Subclass', 'Text'])
df_final.to_csv('FinalDataset/final_dataset.csv', index=False)